##  Training Data

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('./raw_data.csv')

In [44]:
df.head()

,Latitude,Longitude,birth_date,maxtempC,mintempC,totalSnow_cm,sunHour,tempC,windspeedKmph,winddirDegree,...,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC,uvIndex,sunrise,sunset,season
0,49.830870,-119.156620,2018-05-13 18:40:11,25,8,0.0,14.5,16.0,3.0,16.0,...,4.0,17.0,7.0,17.0,5.0,17.0,1,414,1933,spring
1,32.986040,-86.692940,2018-10-13 11:57:35,26,13,0.0,11.6,20.0,7.0,33.0,...,11.0,20.0,9.0,20.0,8.0,20.0,1,550,1716,autumn
2,33.490060,-99.252716,2018-07-30 08:38:32,33,23,0.0,13.3,23.0,36.0,83.0,...,90.0,26.0,22.0,23.0,43.0,26.0,1,548,1939,summer
3,42.903500,-117.783220,2018-11-15 23:37:41,11,1,0.0,8.7,4.0,1.0,267.0,...,10.0,4.0,-5.0,4.0,2.0,4.0,1,744,1728,autumn
4,50.146244,-103.212036,2018-02-02 09:28:53,-17,-24,0.3,3.5,-20.0,4.0,280.0,...,100.0,-20.0,-21.0,-24.0,7.0,-24.0,1,825,1749,winter


In [45]:
df.describe()

,Latitude,Longitude,maxtempC,mintempC,totalSnow_cm,sunHour,tempC,windspeedKmph,winddirDegree,precipMM,...,pressure,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC,uvIndex,sunrise,sunset
count,3442.000000,3442.000000,3442.000000,3442.000000,3442.000000,3442.000000,3442.000000,3442.000000,3442.000000,3442.000000,...,3442.000000,3442.000000,3442.000000,3442.000000,3442.000000,3442.000000,3442.000000,3442.0,3442.000000,3442.000000
mean,40.528542,-99.600947,13.363161,4.693202,0.241778,9.767955,8.843696,12.666182,197.915456,0.188989,...,1017.623765,38.043579,9.293144,1.793144,6.997966,19.517722,7.207728,1.0,619.109820,1816.309994
std,7.225794,13.529726,13.275872,12.350825,1.255309,3.421308,13.073862,7.784317,98.860827,1.109924,...,7.692880,36.664495,13.343071,11.291657,15.117009,11.344868,15.397253,0.0,110.465803,110.336752
min,28.102217,-122.998980,-32.000000,-40.000000,0.000000,1.900000,-40.000000,0.000000,0.000000,0.000000,...,988.000000,0.000000,-39.000000,-42.000000,-47.000000,0.000000,-47.000000,1.0,337.000000,1548.000000
25%,34.204875,-111.250140,3.000000,-3.000000,0.000000,7.400000,0.000000,7.000000,122.250000,0.000000,...,1013.000000,6.000000,-0.000000,-6.000000,-4.000000,11.000000,-4.000000,1.0,533.000000,1730.000000
50%,40.661141,-99.840960,15.000000,6.000000,0.000000,9.900000,10.000000,11.000000,207.000000,0.000000,...,1017.000000,23.000000,10.000000,1.000000,9.000000,17.000000,9.000000,1.0,624.000000,1819.500000
75%,46.771526,-87.880728,24.000000,14.000000,0.000000,12.900000,19.000000,17.000000,281.000000,0.000000,...,1022.000000,72.000000,19.000000,10.000000,19.000000,25.000000,19.000000,1.0,715.000000,1910.000000
max,52.984756,-76.104120,45.000000,34.000000,25.400000,16.800000,42.000000,59.000000,360.000000,33.400000,...,1047.000000,100.000000,43.000000,26.000000,42.000000,80.000000,43.000000,1.0,926.000000,2116.000000


Change 'sunrise' and 'sunset' to integer type 

In [15]:
def parseSunTime(time):
    splitted = time.split(' ')
    hm = splitted[0].split(':')
    parsedTime = int(hm[0])*100 + int(hm[1])
    if splitted[1] == 'PM':
        parsedTime += 1200
            
    return parsedTime
    
df['sunrise'] = df["sunrise"].apply(lambda x: parseSunTime(x))
df['sunset'] = df["sunset"].apply(lambda x: parseSunTime(x))

In [16]:
df.head()

,Latitude,Longitude,birth_date,maxtempC,mintempC,totalSnow_cm,sunHour,tempC,windspeedKmph,winddirDegree,...,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC,uvIndex,sunrise,sunset,season
0,49.830870,-119.156620,2018-05-13 18:40:11,25,8,0.0,14.5,16.0,3.0,16.0,...,4.0,17.0,7.0,17.0,5.0,17.0,1,414,1933,spring
1,32.986040,-86.692940,2018-10-13 11:57:35,26,13,0.0,11.6,20.0,7.0,33.0,...,11.0,20.0,9.0,20.0,8.0,20.0,1,550,1716,autumn
2,33.490060,-99.252716,2018-07-30 08:38:32,33,23,0.0,13.3,23.0,36.0,83.0,...,90.0,26.0,22.0,23.0,43.0,26.0,1,548,1939,summer
3,42.903500,-117.783220,2018-11-15 23:37:41,11,1,0.0,8.7,4.0,1.0,267.0,...,10.0,4.0,-5.0,4.0,2.0,4.0,1,744,1728,autumn
4,50.146244,-103.212036,2018-02-02 09:28:53,-17,-24,0.3,3.5,-20.0,4.0,280.0,...,100.0,-20.0,-21.0,-24.0,7.0,-24.0,1,825,1749,winter


In [17]:
df.columns

Index(['Latitude', 'Longitude', 'birth_date', 'maxtempC', 'mintempC',
       'totalSnow_cm', 'sunHour', 'tempC', 'windspeedKmph', 'winddirDegree',
       'precipMM', 'humidity', 'visibility', 'pressure', 'cloudcover',
       'HeatIndexC', 'DewPointC', 'WindChillC', 'WindGustKmph', 'FeelsLikeC',
       'uvIndex', 'sunrise', 'sunset', 'season'],
      dtype='object')

Prepare training & test dataset.

In [18]:
y = df['season']
x = df.drop(['season', 'birth_date', 'uvIndex', 'cloudcover', 'winddirDegree', 'windspeedKmph'], axis=1)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

### Decision Tree Classifier

In [20]:
dtree = DecisionTreeClassifier().fit(x_train, y_train)

In [21]:
dtree_predictions = dtree.predict(x_test)
accuracy_score(y_test, dtree_predictions)

0.7793263646922184

In [42]:
dtree_matrix = confusion_matrix(y_test, dtree_predictions, labels= ["spring","summer","autumn","winter"])
dtree_matrix

array([[192,  30,  16,   8],
       [ 42, 131,   2,   0],
       [ 27,   4, 118,  24],
       [ 11,   0,  26, 230]])

The accuracy of Decision Tree classifier is 78%. Not bad but not good enough. Based on matrix, the accuracy for spring and autumn is low. 

### Neural Network

In [ ]:
clf = MLPClassifier(hidden_layer_sizes = (10, 10, 10), max_iter=1000, verbose=10)
clf.fit(x_train, y_train)
mlp_y_pred = clf.predict(x_test)

In [23]:
accuracy_score(y_test, mlp_y_pred)

0.6655052264808362

In [25]:
confusion_matrix(y_test, mlp_y_pred, labels= ["spring","summer","autumn","winter"])

array([[232,   4,   2,   8],
       [158,  16,   1,   0],
       [ 49,   0,  87,  37],
       [  2,   0,  27, 238]])

The accuracy of Neural Network is lower than Decision Tree model. I think because there is not enough features for Neural Network. Move on to the next algorithm. 

### Random Forest Classifier

In [47]:
base_model = RandomForestClassifier()
base_model.fit(x_train, y_train)
random_base_pred = base_model.predict(x_test)
accuracy_score(y_test, random_base_pred)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8246225319396051

The accuracy for the Random Forest Classifier is higher than others. I will go on to tuning the hyperparameters. 

In [48]:
rf = RandomForestClassifier()
rf.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [49]:
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 300, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(5, 70, num = 10)]
max_depth.append(None)
min_samples_split = [2, 4, 8, 16]
min_samples_leaf = [1, 2, 4, 8]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [50, 77, 105, 133, 161, 188, 216, 244, 272, 300], 'max_features': ['auto', 'sqrt'], 'max_depth': [5, 12, 19, 26, 33, 41, 48, 55, 62, 70, None], 'min_samples_split': [2, 4, 8, 16], 'min_samples_leaf': [1, 2, 4, 8], 'bootstrap': [True, False]}


In [53]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(x_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   55.3s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.8min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [54]:
rf_random.best_params_

{'n_estimators': 272,
 'min_samples_split': 4,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': None,
 'bootstrap': False}

As a result, the randomized search gave me this set of parameters. 

In [55]:
random_model = rf_random.best_estimator_
best_random_pred = random_model.predict(x_test)
accuracy_score(y_test, best_random_pred)

0.8664343786295006

Now the model has improved accuracy by 4.4%! The accuracy of fine-tuned Random Forest Classifier model is 86.6%. 

I tried 3 classification algorithms to predict seasons based on various geographical data(the number of data is 3442) - Decision tree model, Neural network model, and the Random forest model. The best model is the Random forest with 85% accuracy. 